In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Definir datos del sistema
n=3
Ybus=np.array([(15-35j,-10+20j,-5+15j),(-10+20j,30-60j,-20+40j),(-5+15j,-20+40j,25-55j)],dtype=complex)
B=np.imag(Ybus)
G=np.real(Ybus)

# Valores definidos de potencia
Pdef=np.array([[0.5,-1]])
Qdef=np.array([[-0.6]])

# Definir valores iniciales de voltaje
def rad(deg):
    return deg*np.pi/180

vbusMag=np.array([1.02,1.02,1.02])
vbusAng=np.array([rad(0),rad(0),rad(0)])

def asvoltage(Mag,Ang):
  # Mag: vector de magnitudes de voltaje
  # Ang: vector de ángulos de voltaje
  v=np.zeros((len(Mag),1),dtype=complex)
  for i in range(len(Mag)):
    v[i]=Mag[i]*(np.cos(Ang[i])+1j*np.sin(Ang[i]))
  return v
vbus=asvoltage(vbusMag,vbusAng)
print('Voltajes:\n',vbus)

# Calcular potencias inyectadas a cada bus
S=np.zeros((n,1),dtype=complex)
for i in range(0,n):
    S[i]=vbus[i]*np.conjugate(np.matmul(Ybus[i,:],vbus))

print('Potencia compleja:\n',S)

# Calcular residuos
def residuos(S,Pdef,Qdef):
  Qcal=np.imag(S)
  Pcal=np.real(S)
  dP=np.swapaxes(Pdef,0,1)-np.real(S)[1:3]
  dQ=Qdef-np.imag(S)[2]
  dPQ=np.concatenate((dP,dQ),axis=0)
  return dPQ

dPQ=residuos(S,Pdef,Qdef)
print('Residuos:',dPQ)

Voltajes:
 [[1.02+0.j]
 [1.02+0.j]
 [1.02+0.j]]
Potencia compleja:
 [[1.81188398e-15+3.62376795e-15j]
 [3.62376795e-15+7.24753590e-15j]
 [0.00000000e+00+7.24753590e-15j]]
Residuos: [[ 0.5]
 [-1. ]
 [-0.6]]


Como los residuos son mayores a la tolerancia (supongamos que es 10^-6), s edebe continuar a calcular el Jacobiano y actualizar las variables.

In [ ]:
count=0
pconocida=[2,3]
qconocida=[3]
Qcal=np.imag(S)
Pcal=np.real(S)

while ((max(abs(dPQ))>10^-6) & (count<10)):
  # Calcular Jacobiano

  H=np.zeros((len(pconocida),len(pconocida)))
  for i in range(len(pconocida)):
    for j in range(len(pconocida)):
        k=pconocida[i]-1
        m=pconocida[j]-1
        if k==m:
          H[i][j]=-Qcal[k]-B[k][k]*(vbusMag[k])**2
        else:
          H[i][j]=vbusMag[k]*vbusMag[m]*(G[k][m]*np.sin(vbusAng[k]-vbusAng[m])-B[k][m]*np.cos(vbusAng[k]-vbusAng[m]))

  N=np.zeros((len(pconocida),len(qconocida)))
  for i in range(len(pconocida)):
    for j in range(len(qconocida)):
      k=pconocida[i]-1
      m=qconocida[j]-1
      if k==m:
          N[i][j]=Pcal[k]+G[k][k]*(vbusMag[k])**2
      else:
          N[i][j]=vbusMag[k]*vbusMag[m]*(G[k][m]*np.cos(vbusAng[k]-vbusAng[m])+B[k][m]*np.sin(vbusAng[k]-vbusAng[m]))

  M=np.zeros((len(qconocida),len(pconocida)))
  for i in range(len(qconocida)):
    for j in range(len(pconocida)):
      k=qconocida[i]-1
      m=pconocida[j]-1
      if k==m:
        M[i][j]=Pcal[k]-G[k][k]*(vbusMag[k])**2
      else:
        M[i][j]=-vbusMag[k]*vbusMag[m]*(G[k][m]*np.cos(vbusAng[k]-vbusAng[m])+B[k][m]*np.sin(vbusAng[k]-vbusAng[m]))

  L=np.zeros((len(qconocida),len(qconocida)))
  for i in range(len(qconocida)):
    for j in range(len(qconocida)):
      k=qconocida[i]-1
      m=qconocida[j]-1
      if k==m:
        L[i][j]=Qcal[k]-B[k][k]*(vbusMag[k])**2
      else:
        L[i][j]=vbusMag[k]*vbusMag[m]*(G[k][m]*np.sin(vbusAng[k]-vbusAng[m])-B[k][m]*np.cos(vbusAng[k]-vbusAng[m]))

  J1=np.concatenate((H,N),axis=1)
  J2=np.concatenate((M,L),axis=1)
  J=np.concatenate((J1,J2),axis=0)
  print('Jacobiano: \n',J)

  # Calcular variaciones
  dDV=np.linalg.solve(J, dPQ)
  print('Variaciones: \n',dDV)

  # Actualizar voltajes
  for i in range(n-1):
    vbusAng[pconocida[i]-1]+=dDV[i]
  for i in range(len(qconocida)):
    vbusMag[qconocida[i]-1]=vbusMag[qconocida[i]-1]*(1+dDV[i+n-1])
  vbus=asvoltage(vbusMag,vbusAng)
  print('Voltajes: \n',vbusMag)
  print('Ángulos:\n',vbusAng)

  #Actualizar potencias
  for i in range(0,n):
    S[i]=vbus[i]*np.conjugate(np.matmul(Ybus[i,:],vbus))
  Qcal=np.imag(S)
  Pcal=np.real(S)
  print('Potencias:',S)

  #Calular residuos
  dPQ=residuos(S,Pdef,Qdef)
  print('Residuos: \n',dPQ)

  count+=1
  print('Iteración:',count)


Jacobiano: 
 [[ 62.424 -41.616 -20.808]
 [-41.616  57.222  26.01 ]
 [ 20.808 -26.01   57.222]]
Variaciones: 
 [[-0.00795303]
 [-0.01641642]
 [-0.01505548]]
Voltajes: 
 [1.02       1.02       1.00464341]
Ángulos:
 [ 0.         -0.00795303 -0.01641642]
Potencias: [[ 0.49714922+0.07083377j]
 [ 0.49575791+0.53796447j]
 [-0.98350465-0.58743336j]]
Residuos: 
 [[ 0.00424209]
 [-0.01649535]
 [-0.01256664]]
Iteración: 1
Jacobiano: 
 [[ 61.88603553 -41.16143608 -20.14708614]
 [-40.81453056  56.09939487  24.24920513]
 [ 20.84089718 -26.21621442  54.92452815]]
Variaciones: 
 [[-0.00026815]
 [-0.00035995]
 [-0.00029886]]
Voltajes: 
 [1.02       1.02       1.00434317]
Ángulos:
 [ 0.         -0.00822119 -0.01677637]
Potencias: [[ 0.50976795+0.0709557j ]
 [ 0.49999934+0.55125208j]
 [-0.99999455-0.59999503j]]
Residuos: 
 [[ 6.64282688e-07]
 [-5.45360663e-06]
 [-4.96983098e-06]]
Iteración: 2
